In [15]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth
from dotenv import load_dotenv
from tqdm import tqdm
import os

In [16]:
load_dotenv(os.path.join(os.path.expanduser('~'), '.neo4j_config.env'))
USER = os.environ.get('SPOKE_USER')
PSW = os.environ.get('SPOKE_PSW')
URI = os.environ.get('SPOKE_URI')

In [17]:
%%time

query = """
        MATCH(f:Food)
        RETURN DISTINCT f.identifier AS f_id, f.name AS f_name
"""

auth = basic_auth(USER, PSW)
sdb = GraphDatabase.driver(URI, auth=auth)

food_list = []
with sdb.session() as session:
    with session.begin_transaction() as tx:
        result = tx.run(query)
        for index, row in enumerate(result):
            food_list.append((row["f_id"], row["f_name"]))
            
            
food_df = pd.DataFrame(food_list, columns=["food_id", "food_name"])


CPU times: user 343 ms, sys: 46.4 ms, total: 389 ms
Wall time: 9.24 s


In [18]:
food_df.to_csv("../../data/foodON.csv", index=False, header=True)
